In [2]:
import numpy as np
import pandas as pd
from Machine_Learning.elo import *
import datetime as dt
from Machine_Learning import kristaps
from __future__ import division

reload(kristaps)

<module 'Machine_Learning.kristaps' from 'Machine_Learning/kristaps.pyc'>

In [7]:
# Read in data
filename = 'data/contrived.pickle'
data = pd.read_pickle(filename)
data = data[:-5000]
train = data.iloc[:-300]
test = data.iloc[-300:]

In [8]:
Y_train = train['A Win']
X_train = train[['Fran_5_PTS','Opp_5_PTS']]

Y_test = test['A Win']
X_test = test[['Fran_5_PTS','Opp_5_PTS']]

# SVM

In [42]:
from sklearn import svm
SVM_model = svm.SVC().fit(X_train,Y_train)
prob = np.sum(SVM_model.predict(X_test) == Y_test)/len(Y_test)
print prob

# Holy slow

0.63


# Logistic Regression

In [41]:
from sklearn import linear_model as lm

lam = 10**2

sk_logreg = lm.LogisticRegression(C=np.abs(1/lam))
sk_logreg.fit(X_train, Y_train)
print (sk_logreg.predict(X_test) == Y_test).mean()

# Surpsingly(maybe not) good, very fast

0.63


# Ridge Regression 

In [24]:
from sklearn.linear_model import Ridge as ri

skRidge = ri()
skRidge.fit(X_train, Y_train)
#Print this to see its not empty
#print (skRidge.predict(X_test))
print (skRidge.predict(X_test) == Y_test).mean()

0.0


# Gradient Boost 

In [45]:
from sklearn.ensemble import GradientBoostingClassifier as GBC

gbc = GBC(max_leaf_nodes=200, min_weight_fraction_leaf=0.5,min_samples_split=54).fit(X_train,Y_train)
print (np.mean(np.abs(gbc.predict(X_test) - Y_test)))

# Fine tuning seems to have minimal effect

0.37


# MLP

In [39]:
from sklearn.neural_network import MLPClassifier as mlp

gbm = mlp(solver="lbfgs", activation='tanh', early_stopping=True).fit(X_train,Y_train)
print (np.mean(np.abs(gbm.predict(X_test) - Y_test)))

# Super slow, fine tuning doesnt seem to help much

0.37


# Random Forests

In [25]:
from sklearn.ensemble import RandomForestClassifier as RFC

rfc = RFC(max_leaf_nodes=100000, min_samples_split=3, n_estimators =80).fit(X_train,Y_train)
print (np.mean(np.abs(rfc.predict(X_test) - Y_test)))

# Fine tuned parameters, this is the best I could get

0.443333333333


# Decision Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier as DTC

dtc = DTC(criterion='entropy').fit(X_train,Y_train)
print (np.mean(np.abs(dtc.predict(X_test) - Y_test)))

0.436666666667


# Algorithms we chose not to use

### Nearest neighbor

Due to the fact that we are not performing any kind of cluster analysis, we have decided the NN-classifying and NN-regression algorithms are not useful. 

### Gaussian discriminant analysis

Similar to the issues with nearest neighbor, since we are not doing any kind of classification, GDA will not be useful with our data set or our project.

### Mixture models with latent variables (train with EM)

Because there is not an unknown distribution in our dataset, mixture models along with EM will not work with our data set.


### Kalman Filters

Our data is not a true time series because we are not interested in how teams' overall performances changes over time. Games are decided strictly by winners and losers, so there is not any measurable error or noise. Because of this we have determined that it will not be useful in our methods. 

### AR, MA, ARMA, ARIMA time series models

Because these models are trying to describe certain time-varying processes of a time series, they will not be helpful in trying to predict wins and losses of specific games. 